In [1]:
# imports #
import random

import time
import datetime
import matplotlib.pyplot as plt
import pickle
import numpy as np

import math
import statistics as st
from collections import defaultdict
from client.info_messages import TradeTick
from client.exec_messages import Trade

from cointegration_analysis import engle_granger_two_step_cointegration_test
from cointegration_analysis import estimate_long_run_short_run_relationships

In [1]:
from IPython.display import clear_output

import sys
sys.path.append('..')

from client.synchronous_client import InfoOnly
from client.synchronous_client import Exchange
from client.info_messages import PriceBook, PriceVolume
i = InfoOnly(host='')
i.connect()

from IPython.display import clear_output

import logging
logger = logging.getLogger('client')
logger.setLevel('ERROR')
e = Exchange(host='')
a = e.connect('', '')

2021-08-28 13:30:31,572 [asyncio   ] [MainThread  ] Using selector: SelectSelector
2021-08-28 13:30:31,620 [client    ] [Thread-6    ] background thread started


Exception: Unable to connect to the exchange

In [4]:
# read data from pickle files #
def read_data(timestamps_pckl, stock_values_pckl):
    with open(timestamps_pckl, 'rb') as f:
        timestamps = pickle.load(f)
    with open(stock_values_pckl, 'rb') as f:
        stock_values = pickle.load(f)
    return timestamps, stock_values

In [5]:
# Load data into timestmaps and stock_values.
timestamps, stock_values = read_data('timestamps.pckl', 'stock_values.pckl')

# Create a new dictionary called log_stock_values with a similar structure to 
# stock_values, but containing the log of these instead.
log_stock_values = {name: [math.log(v) for v in stock_values[name]] for name in stock_values}

our_list = ['AIRBUS', 'ALLIANZ', 'ASML', 'LVMH', 'SAP', 'SIEMENS', 'TOTAL', 'UNILEVER']

In [6]:
def best_pairs(list_of_stocks, stock_values):
    
    """Best_pairs function finds the best trading pairs out of a list of names of stocks as well as their stock returns using 
    the engle_granger_two_step_cointegration_test function with log returns. 
    The best trading pairs are those with the most negative dfstat, thus those with greater cointegration.
    
    
    Input: a list of names of stocks, a dictionary that has as keys the names of the stock and as values a list of their returns 
    Output: a dictionary that has as keys the names of the best pairs and as values their dfstat 
    """
    
#     #tranform the normal returns to log returns 
#     log_stock_values = {name: [math.log(v) for v in stock_values[name]] for name in stock_values}

    dfstat_dic, p_value_dic = (defaultdict(float) for i in range(2))
    dfstat_best_dic = (defaultdict(float))

    #double iteration through all 64 possible pairs
    for name1 in list_of_stocks:
        for name2 in list_of_stocks:
            #check if self
            if name1!=name2:
                dfstat, p_value = engle_granger_two_step_cointegration_test(log_stock_values[name1],log_stock_values[name2])
                #only when p-value < 0.01
                if p_value<0.01:
                        key=(name1,name2)
                        key2=(name2,name1)
                        #check if opposite pair exists already in dictionary
                        if key2 in dfstat_dic:
                            #check which one has the most negative dfstat and keep only these
                            if dfstat<dfstat_dic[key2]:
                                dfstat_dic[key]+=dfstat
                                dfstat_best_dic[key]+=dfstat
                            else:
                                dfstat_dic[key]+=dfstat
                                if key2 not in dfstat_best_dic:
                                    dfstat_best_dic[key2]+=dfstat_dic[key2]
                        else:
                            dfstat_dic[key]+=dfstat
                            p_value_dic[key]+=p_value

    return dfstat_best_dic

In [7]:
best=best_pairs(our_list,log_stock_values)

C:\Users\Oliwia1596\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [8]:
best

defaultdict(float,
            {('LVMH', 'ALLIANZ'): -79.83432299252725,
             ('SAP', 'ASML'): -76.09277346657072,
             ('SIEMENS', 'AIRBUS'): -52.11949423881058,
             ('TOTAL', 'UNILEVER'): -39.42636926693103})

In [9]:
# list of names of our pairs
list_of_pairs=[]
for key in best.keys():
    list_of_pairs.append(key)

print(list_of_pairs)
print(list_of_pairs[3])

[('LVMH', 'ALLIANZ'), ('SAP', 'ASML'), ('SIEMENS', 'AIRBUS'), ('TOTAL', 'UNILEVER')]
('TOTAL', 'UNILEVER')


In [10]:
def values_of_best_pairs(pairs,stock_values):
    """Values_of_best_pairs function calculates the c,gamma,alpha and z of each pair of stocks of a given list. Each element of the list
    is in the form of 'Name1-Name2'. The gammas are calculated using the estimate_long_run_short_run_relationships() function
    
    
    Input: A list of pairs where each element is in the form of 'Name_of_the_first_stock-Name_of_the_second_stock', and a dictionary that
    has as keys the names of the stock and as values a list of their returns.
    Output: A dictionary that has as keys the pairs of the stocks in the form 'Name_of_the_first_stock-Name_of_the_second_stock' and
    as values their respective gamma.
    """
    
    c_dic, gamma_dic, alpha_dic = (defaultdict(float) for i in range(3))
    z_dic = defaultdict(list)

    #iterate through pairs and find (c) , gamma , (alpha) , (z) for each one
    for pair in pairs:
            c , gamma , alpha , z = estimate_long_run_short_run_relationships(log_stock_values[pair[0]], log_stock_values[pair[1]])
            key=(pair[0],pair[1])
            c_dic[key]+=c
            gamma_dic[key]+=gamma
            alpha_dic[key]+=alpha
            z_dic[key].append(z)
    

    return c_dic,gamma_dic,alpha_dic, z_dic

In [11]:
c,gamma,alpha,z= values_of_best_pairs(list_of_pairs,log_stock_values)


In [12]:
print(c)

defaultdict(<class 'float'>, {('LVMH', 'ALLIANZ'): 1.1445122982673936, ('SAP', 'ASML'): -1.0113536576920978, ('SIEMENS', 'AIRBUS'): 1.2669276350298588, ('TOTAL', 'UNILEVER'): 2.662989564576037})


In [13]:
print(gamma)

defaultdict(<class 'float'>, {('LVMH', 'ALLIANZ'): 0.9126035438648543, ('SAP', 'ASML'): 1.0511929013538648, ('SIEMENS', 'AIRBUS'): 0.8205285711557515, ('TOTAL', 'UNILEVER'): 0.40623135619686934})


In [14]:
print(alpha)

defaultdict(<class 'float'>, {('LVMH', 'ALLIANZ'): -0.11968252555949839, ('SAP', 'ASML'): -0.10897368609345703, ('SIEMENS', 'AIRBUS'): -0.05132599133393867, ('TOTAL', 'UNILEVER'): -0.02971746726142706})


In [15]:
print(z)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
def get_latest_trade(id):
    """This get_latest_trade() function returns the latest tradetick of the stock 
    
    Input:Name of a stock
    Output:A tuple with the name of the stock and the last trade tick
    """
    
    trade=e.poll_new_trade_ticks(id)
# if poll_new_trade_ticks was called too early and no other trades happened meantime, it will return an empty list #
    if trade!=[]:
        return trade[-1]
    if trade==None:
        return []




In [17]:
def get_price(id):
    """This get_price() function returns the price of the trade
    
    Input:The name of a stock
    Output:The price of the stock at that moment
    """
### we put in while in order not to return an empty list ###
    while True:
        trade = e.get_trade_tick_history(id)
        if trade != []:
            return trade[-1]
    


In [18]:
def get_volume(trade):
    """This get_volume() function returns the volume of the trade
    
    Input:A trade of a stock
    Output:The volume of the stock at that moment
    """
    
    if trade!=None:
        return trade.volume
    else:
        return trade


In [19]:
def fair_price(pair,price2):
    """This fair_price() function returns the fair price of a stock translated to the others
    
    Input:The pair and the actual price of the first stock 
    Output:The fair price
    """
    price1 = c[pair] + gamma[pair]*math.log(price2)
    fair_price = math.exp(price1)
    return fair_price



In [20]:
def ratio_of_volume(pair,price1,price2):
    """This ratio_of_volume() function returns the hedging ratio for the conversion of the volumes 
    
    Input:Pair, price of stock1, and price of stock2
    Output:The hedging ratio
    """
    
    return (gamma[pair]*(price1/price2))



In [21]:
def update_trade_list(trade_list):
    result = trade_list
    for trade in trade_list:
        if trade != None:
            last_trade = get_latest_trade(trade.instrument_id)
            if last_trade != None:
                result.remove(trade)
                result.append(last_trade)
    return result

def name_price(trade_list):
    dict = {}
    for stock in trade_list:
        dict[stock.instrument_id] = stock.price
    return dict

def check_limit(instrument_id, volume, side_of_trade):
    #check if volume>800 limit
    if volume>800:
        volume=800
    
    pos=e.get_positions()
    
    #check if there is already a position for this instrument 
    if instrument_id not in pos.keys():
        if volume > 500:
            volume=500
        
    else:
        for p in pos:
            if p==instrument_id:
                #if the trade is on ask side check negative 500 limit else check positive 500 limit
                if side_of_trade=='ask':
                    if pos[p]-volume<-500:
                        volume = 500 - abs(pos[p])
                    else:
                        volume = 500 - abs(pos[p])
                else:
                    if pos[p]+volume>500:
                        volume = 500 - abs(pos[p])
                    else:
                        volume = 500 - abs(pos[p])
    return volume

def best_bid(pb):
    if not pb.bids:
        return None

    best_bid = max(pb.bids, key=lambda x: x.price)
    return best_bid

def best_ask(pb):
    if not pb.asks:
        return None

    best_ask = min(pb.asks, key=lambda x: x.price)
    return best_ask